In [2]:
import pandas as pd
from pymongo import MongoClient

MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "CS3203"
COLLECTION_CATEGORIES = "categories"
COLLECTION_COMPLAINTS = "complaints"

categories = {
    "Public Safety",
    "Financial",
    "Transportation",
    "Technology",
    "Food Services",
    "Housing",
    "Employment",
    "Retail",
    "Healthcare",
    "Social Services",
    "Education",
    "Environmental",
    "Noise",
    "Infrastructure",
    "Others"
}

client = MongoClient(MONGO_URI)
db = client[DB_NAME]
categories_collection = db[COLLECTION_CATEGORIES]

data = [
    {"category": "Public Safety", "color": "#FF5733"},  # Vibrant Red-Orange
    {"category": "Financial", "color": "#2E86C1"},  # Deep Blue
    {"category": "Transportation", "color": "#1ABC9C"},  # Teal
    {"category": "Technology", "color": "#8E44AD"},  # Purple
    {"category": "Food Services", "color": "#E67E22"},  # Warm Orange
    {"category": "Housing", "color": "#27AE60"},  # Forest Green
    {"category": "Employment", "color": "#F1C40F"},  # Bright Yellow
    {"category": "Retail", "color": "#D35400"},  # Burnt Orange
    {"category": "Healthcare", "color": "#E74C3C"},  # Bold Red
    {"category": "Social Services", "color": "#5DADE2"},  # Sky Blue
    {"category": "Education", "color": "#9B59B6"},  # Soft Purple
    {"category": "Environmental", "color": "#2ECC71"},  # Fresh Green
    {"category": "Noise", "color": "#F39C12"},  # Golden Yellow
    {"category": "Infrastructure", "color": "#34495E"},  # Dark Gray-Blue
    {"category": "Others", "color": "#95A5A6"}  # Neutral Gray
]

categories_collection.drop()
categories_collection.insert_many(data)

print("Categories inserted successfully!")

raw_df = pd.read_csv("./data/sentiment_scored_2023_data.csv")
raw_df = raw_df[raw_df["Intent Category"] == "Direct Complaint"]
clean_df = pd.DataFrame()
clean_df["id"] = raw_df["name"]
clean_df["title"] = raw_df["title"]
clean_df["description"] = raw_df["selftext"].fillna("")
clean_df["category"] = raw_df["Domain Category"].apply(lambda i: i if i in categories else "Others")
clean_df["date"] = pd.to_datetime(raw_df["created_utc"])
clean_df["sentiment"] = raw_df["sentiment_title_selftext_polarity"]
clean_df["url"] = raw_df["name"].apply(lambda i: f"https://www.reddit.com/r/singapore/comments/{i}")
clean_df["source"] = "Reddit"
clean_df = clean_df.dropna()

client = MongoClient(MONGO_URI)  
db = client[DB_NAME]  
complaints_collection = db[COLLECTION_COMPLAINTS]  

data = clean_df.to_dict(orient="records")

complaints_collection.drop()
complaints_collection.insert_many(data)
complaints_collection.create_index([("title", "text"), ("description", "text")])

print("Data successfully inserted into MongoDB!")

Categories inserted successfully!
Data successfully inserted into MongoDB!
